## EDA

#### Primero importamos las librerias requeridas

In [49]:
import numpy as np
import pandas as pd
import seaborn as sbn

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics




#### Cargamos los Datasets

In [50]:
df_test = pd.read_parquet('./proyecto_ML/test.parquet')
df_train = pd.read_parquet('./proyecto_ML/train.parquet')

#### Obvervamos las Columnas de los Datasets

In [51]:
df_test.head(5)

,id,url,region,region_url,type,sqfeet,beds,baths,cats_allowed,dogs_allowed,...,wheelchair_access,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,image_url,description,lat,long,state
0,7037609789,https://annarbor.craigslist.org/apa/d/wixom-ho...,ann arbor,https://annarbor.craigslist.org,manufactured,1344,3,2.0,0,0,...,0,0,0,w/d in unit,off-street parking,https://images.craigslist.org/00M0M_iNczP1nzIL...,"OPEN HOUSE TODAY! APPLY THIS WEEK, PUT A HOLDI...",42.5333,-83.5763,mi
1,7032406876,https://vermont.craigslist.org/apa/d/randolph-...,vermont,https://vermont.craigslist.org,apartment,1050,2,1.0,0,0,...,0,0,0,w/d hookups,off-street parking,https://images.craigslist.org/00L0L_ecirmYBIzL...,"Think of it, you'll be first to get your mail....",43.9393,-72.5538,vt
2,7037022682,https://annarbor.craigslist.org/apa/d/ann-arbo...,ann arbor,https://annarbor.craigslist.org,apartment,1150,2,2.0,1,1,...,1,0,0,w/d in unit,carport,https://images.craigslist.org/00e0e_dPln2xjo9g...,One of Ann Arbor's most luxurious apartment co...,42.2492,-83.7712,mi
3,7048681802,https://fortcollins.craigslist.org/apa/d/fort-...,fort collins / north CO,https://fortcollins.craigslist.org,apartment,1280,2,2.5,1,1,...,0,0,0,w/d in unit,attached garage,https://images.craigslist.org/00L0L_jlektT5cSd...,"Specials! Move in before January 16th, 2020 an...",40.5501,-105.0350,co
4,7043597870,https://charlottesville.craigslist.org/apa/d/c...,charlottesville,https://charlottesville.craigslist.org,apartment,783,2,1.0,1,1,...,0,0,0,laundry on site,None,https://images.craigslist.org/00D0D_cXa4KbZ6ox...,Barracks West Apartments & Townhomes in Charlo...,38.0936,-78.5611,va


In [52]:
df_train.head(5)

,id,url,region,region_url,price,type,sqfeet,beds,baths,cats_allowed,...,wheelchair_access,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,image_url,description,lat,long,state
0,7048013474,https://boise.craigslist.org/apa/d/very-nice-b...,boise,https://boise.craigslist.org,1350,house,1200,2,2.0,1,...,0,0,0,w/d in unit,detached garage,https://images.craigslist.org/00B0B_cPiJMEheZe...,Super cute row house in the Boise bench area. ...,43.5851,-116.2250,id
1,7043931179,https://cosprings.craigslist.org/apa/d/colorad...,colorado springs,https://cosprings.craigslist.org,1115,apartment,694,1,1.0,1,...,0,0,0,w/d in unit,carport,https://images.craigslist.org/00R0R_5XAoSKvfrz...,Windtree Apartment Homes currently has a spaci...,38.9137,-104.7800,co
2,7048254516,https://norfolk.craigslist.org/apa/d/virginia-...,norfolk / hampton roads,https://norfolk.craigslist.org,1129,apartment,900,2,2.0,0,...,0,0,0,w/d hookups,off-street parking,https://images.craigslist.org/00f0f_3ZbTFrsHpZ...,Call Today! show contact info Indian Lakes ...,36.7922,-76.1643,va
3,7041032577,https://phoenix.craigslist.org/nph/apa/d/phoen...,phoenix,https://phoenix.craigslist.org,1580,house,1469,3,2.0,1,...,0,0,0,w/d in unit,None,https://images.craigslist.org/00f0f_aXV5Dkd5qk...,Cody Anderson Two Brothers Realty License #: S...,33.5623,-112.0560,az
4,7048588701,https://nashville.craigslist.org/apa/d/antioch...,nashville,https://nashville.craigslist.org,995,apartment,700,1,1.0,1,...,0,0,0,w/d in unit,carport,https://images.craigslist.org/00606_gYOGKClOHv...,To schedule a tour We now book our tour appoin...,36.0595,-86.6592,tn


#### Se retiran las Columnas que se consideran innecesarias para el entrenamiento del modelo, en el caso de las URL, buen parte de ellas (si no todas) dirigen a un error 404, tampoco vamos a usar la latitud y la longitud, también filtramos el precio objetivo con una función.

In [53]:
df_train['target'] = df_train['price'].apply(lambda x: 1 if x < 999 else 0)
df_test['target'] = None

df_train.drop(['id','url','state','region_url', 'laundry_options', 'parking_options', 'image_url','lat', 'long', 'description' ], axis= 1, inplace= True)
df_test.drop(['id','url','state','region_url', 'laundry_options', 'parking_options', 'image_url','lat', 'long', 'description' ], axis= 1, inplace= True)

In [54]:
df_train = pd.get_dummies(df_train)
df_test = pd.get_dummies(df_test)

#### Usamos las Columnas que estan en ambos datasets

In [55]:
in_both = set(df_train.columns).intersection(set(df_test.columns))


In [56]:
X = df_train[in_both]
y = df_train.target

C:\Users\UsuarioPc\AppData\Local\Temp\ipykernel_1804\2867608284.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  X = df_train[in_both]


In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [58]:
clf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state = 42)
clf.fit(X_train,y_train)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [61]:
y_train_prediction = clf.predict(X_train)
y_test_prediction = clf.predict(X_test)

#### Accuracy Score

In [60]:
train_acc = metrics.accuracy_score(y_train, y_train_prediction)
test_acc = metrics.accuracy_score(y_test, y_test_prediction)

train_acc, test_acc

(0.9809468424226759, 0.9310733870332957)

In [64]:
X_final = df_test[list(in_both)]
y_final = clf.predict(X_final)
prediction = pd.DataFrame(y_final,columns=["objetivo"])
prediction.to_csv("diegomoos.csv", index=False)

In [65]:
prediction.head()

,objetivo
0,0
1,0
2,0
3,0
4,0


In [66]:
prediction.tail()

,objetivo
38493,0
38494,1
38495,0
38496,0
38497,0
